In [10]:
import requests
import pandas as pd

from datetime import timedelta, datetime
import pendulum

from prefect import Client
from prefect.schedules import IntervalSchedule
from prefect.environments.storage import Docker


from extractMOCData import CONFIG as cfg
from extractMOCData.moc_data import scrape_tsxmoc_fl

In [11]:
pre_client = Client()
project_nm = "MOC"

In [12]:
try:
    proj_id = pre_client.create_project(project_name=project_nm)
except Exception as err:
    print(err.with_traceback)

<built-in method with_traceback of ClientError object at 0x7f7092a1ef30>


In [15]:
scrape_tsxmoc_fl.storage = Docker(dockerfile="/home/ilivni/MOC/Dockerfile")
scrape_tsxmoc_fl.register(project_name=project_nm)

[2020-04-02 05:28:33,036] INFO - prefect.Docker | Building the flow's Docker storage...
Step 1/15 : FROM python:3.7
 ---> 879165535a54
Step 2/15 : COPY . /extractMOCData
 ---> 32d11ec6fdc3
Step 3/15 : WORKDIR /extractMOCData
 ---> Running in a137f2d8b415
Removing intermediate container a137f2d8b415
 ---> 86e75c456f1c
Step 4/15 : RUN apt-get update && apt-get install -y
 ---> Running in 0b152279c19d
Get:1 http://security.debian.org/debian-security buster/updates InRelease [65.4 kB]
Get:2 http://deb.debian.org/debian buster InRelease [122 kB]
Get:3 http://deb.debian.org/debian buster-updates InRelease [49.3 kB]
Get:4 http://security.debian.org/debian-security buster/updates/main amd64 Packages [186 kB]
Get:5 http://deb.debian.org/debian buster/main amd64 Packages [7907 kB]
Get:6 http://deb.debian.org/debian buster-updates/main amd64 Packages [7380 B]
Fetched 8336 kB in 4s (2199 kB/s)
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information..

ValueError: Your docker image failed to build!  Your flow might have failed one of its deployment health checks - please ensure that all necessary files and dependencies have been included.

In [28]:
Mschedule = IntervalSchedule(
    start_date=datetime.utcnow() + timedelta(seconds=1),
    interval=timedelta(minutes=1),
)
schedule

In [24]:
def retrieve_url(url='https://api.tmxmoney.com/mocimbalance/en/TSX/moc.html'):
    
    html = requests.get(url).content
    df_list = pd.read_html(html, header=[0], displayed_only=False)
    moc_df = df_list[-1]
    
    moc_df["moc_date"] = dt.datetime.today().strftime('%Y%m%d')
    return moc_df

In [25]:
moc_df = moc_to_df()

In [26]:


today = pd.Timestamp('today')
moc_df.to_csv(f"moc_{today:%Y%m%d}.csv", index=False)

In [11]:
moc_df.columns


Index(['Symbol', 'Imbalance Side', 'Imbalance Size',
       'Imbalance Reference Price', 'moc_date'],
      dtype='object')

In [18]:
moc_df["Symbol"] =moc_df["Symbol"].fillna("NA")

In [21]:
moc_df[moc_df["Symbol"].isnull()]

,Symbol,Imbalance Side,Imbalance Size,Imbalance Reference Price,moc_date


In [ ]:
moc_df.to_json(orient="records")